In [ ]:
%%spark
// val dfProduct = spark.read.synapsesql("edwdb.cso.DimProduct")
// dfProduct.write.mode("overwrite").format("delta").save("/data/contoso/DimProduct/")

// // You can set tables used in this example by installing contoso sample database from below location in SQL pool database
// // https://github.com/microsoft/sql-server-samples/tree/master/samples/databases/contoso-data-warehouse

In [ ]:
%%spark
// import org.apache.spark.sql.functions._
// val dfFactOnlineSales = spark.read.synapsesql("edwdb.cso.FactOnlineSales")
// val dfFactOnlineSales1 = dfFactOnlineSales.withColumn("SaleYear", year(dfFactOnlineSales("DateKey")))
// dfFactOnlineSales1.write.mode("overwrite").format("delta").partitionBy("SaleYear").save("/data/contoso/FactOnlineSales/")

In [ ]:
dfProduct = spark.read.format("delta").load("/data/contoso/DimProduct/")
dfProduct.createOrReplaceTempView("DimProduct")

dfFactOnlineSales = spark.read.format("delta").load("/data/contoso/FactOnlineSales/")
dfFactOnlineSales.createOrReplaceTempView("FactOnlineSales")

In [ ]:
dfJoinedSale = dfProduct.join(dfFactOnlineSales, dfProduct.ProductKey == dfFactOnlineSales.ProductKey) 
dfJoinedSale = dfJoinedSale.filter(dfJoinedSale.SaleYear == 2009)
# display(dfJoinedSale)

dfJoinedSale.explain() # only physical plan
# dfJoinedSale.explain(True) # all plans including logical and physical plan

In [ ]:
dfAggregatedSale = dfJoinedSale.groupBy('BrandName').agg({'SalesAmount':'sum'})
# display(dfAggregatedSale)
dfAggregatedSale.explain()

In [ ]:
%%sql
EXPLAIN
SELECT  SUM(f.SalesAmount) AS sales_by_brand_amount
,       p.BrandName
FROM    FactOnlineSales AS f
JOIN    DimProduct      AS p ON f.ProductKey = p.ProductKey
GROUP BY p.BrandName

In [ ]:
print(spark.conf.get("spark.sql.autoBroadcastJoinThreshold")) # 26214400 - 26 MB

# You can use join hint, as below, to instruct Optimizer to broadcast for the specified database
# df1.join(broadcast(df2), Seq("id")).explain #broadcast join hint

In [ ]:
%%sql
SELECT  SUM(f.SalesAmount) AS sales_by_brand_amount
,       p.BrandName
FROM    FactOnlineSales AS f
JOIN    DimProduct      AS p ON f.ProductKey = p.ProductKey
GROUP BY p.BrandName

In [ ]:
# Adaptive Query Execution https://spark.apache.org/docs/latest/sql-performance-tuning.html#adaptive-query-execution
print(spark.conf.get("spark.sql.adaptive.enabled"))